## 네이버 지식인 크롤링


## 필요한 라이브러리 설치
* 아나콘다 사용시 다음의 프롬프트 창을 열어 conda 명령어로 설치합니다.
* pip 사용시 아래에 있는 명령어를 터미널로 설치합니다.
<img src="https://i.imgur.com/Sar4gdw.jpg">

### Selenium
* `conda install -c anaconda selenium`
* [Selenium :: Anaconda Cloud](https://anaconda.org/anaconda/selenium)

* pip 사용시 : `pip install selenium`

### BeautifulSoup
* `conda install -c anaconda beautifulsoup4`
* [Beautifulsoup4 :: Anaconda Cloud](https://anaconda.org/anaconda/beautifulsoup4)

* pip 사용시 : `pip install beautifulsoup4`

### tqdm
* `conda install -c conda-forge tqdm`
* [tqdm/tqdm: A Fast, Extensible Progress Bar for Python and CLI](https://github.com/tqdm/tqdm)
* `pip install tqdm`

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

import time
from tqdm import trange
from pandas import read_excel

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

keywords = ['외식', '식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']

driver = webdriver.Chrome(options=options)

interval = 4
def get_postURLs(keyword, pnum):

    url = f"https://m.kin.naver.com/mobile/search/searchList.nhn?cs=utf8&query={keyword}&startDate=2019-07-01&endDate=2020-06-30"
    
    try:
        # 키워드 지식인 검색
        driver.get(url)
        time.sleep(interval)
        
        # Scrap할 페이지 수만큼 열기
        pages = pnum // 20
        for i in trange(pages): # 페이지별 20개 Posts
            try:
                driver.find_element_by_xpath("//*[@id='moreContainer']/div/a[1]").click()
                time.sleep(interval)
            except:
                continue
    except:
        print (keyword_url)

    # postURL 뒷부분 스크랩
    soup = bs(driver.page_source, 'html.parser')
    posts = soup.find_all('div', attrs={"class":"text_area"})

    # 온전한 postURL 만들기
    post_base_url = "https://m.kin.naver.com"
    post_urls = []

    for post in posts:
        post_url = post_base_url + post.find('a').get('href')
        post_urls.append(post_url)

    df_postURLs = pd.DataFrame({"url": post_urls})
    
    return df_postURLs

# 지식인 키워드 검색 Main 호출
pnum = 10000
for keyword in keywords:
    try:
        df_postURLs = get_postURLs(keyword, pnum)
        filename = "./urls/" + "kin-posts_" + keyword.replace(" ","") + ".csv"
        df_postURLs.to_csv(filename, index=False)
    except:
        continue

driver.close()
driver.quit()
    

100%|███████████████████████████████████████████████████████████████████| 500/500 [05:20<00:00,  1.56it/s]
